In [1]:
# collect data for training 
# handling the data in memory import pandas 
# data to get our data from... 
import MySQLdb 
import csv 
import pandas
import datetime
import os
import shutil

In [2]:
def CleanAndMakeDir(destinationFolder):
    if (os.path.isdir(destinationFolder)):
        shutil.rmtree(destinationFolder)
    os.mkdir(destinationFolder)

In [3]:
def crossesBarrier(event1, event2):
    if event1.time() < datetime.time(4,0) and event2.time() > datetime.time(4,0):
        return True
    elif (event1.date() < event2.date() and 
        (event1.time() < datetime.time(4,0) or event2.time() > datetime.time(4,0))):
        return True
    else:
        return False

In [4]:
def stale(last_time_tutor_accpted_tick, currTime, n):
    time_since_tutor_accpted_last_tick = currTime - last_time_tutor_accpted_tick
    stale_minutes = str(n) + " minutes"
    if time_since_tutor_accpted_last_tick > pandas.Timedelta(stale_minutes):
        return True
    else:
        return False

In [5]:
def removeStaleTutors(tutorOnDuty, currTime, n):
    staleTutors = set()
    for tutor in tutorOnDuty:
        if stale(tutorOnDuty[tutor], currTime, n):
            staleTutors.add(tutor)
            
    for tutor in staleTutors:
        del tutorOnDuty[tutor]

In [6]:
db = MySQLdb.connect(host="localhost", # your host, usually localhost 
user="root", # your username 
passwd="", # your password 
db="autograder")


In [27]:
def GetInfoFromQueue(queueId, destinationFolder):
    events = pandas.read_sql("""select TicketEvent.type,
                        EventBase.time, EventBase.user_id,
                        Ticket.queue_id, TicketEvent.ticket_id,
                        Course.shortName, Queue.course_id
                        from TicketEvent, EventBase, Ticket, Course, Queue
                        where TicketEvent.id = EventBase.id and
                        TicketEvent.ticket_id = Ticket.id and
                        Ticket.queue_id = """ + str(queueId)
                        + """ and Queue.id = """ + str(queueId) + """ and 
                             Course.id = Queue.course_id order by EventBase.time;""", db)
    
    #creates a results dataframe and fills in the values for position on queue, and wait time
    results = pandas.DataFrame(columns=('ticket_id', 'position', "created_at",
                                        "accepted_at", "diff","tutors_on_duty", "day_of_week", "time_of_day", "course",
                                        "avg_wait_for_day", "daily_ticket_count", "tutor_id", "time_to_resolve"))
    counter = 0 
    index = 0
    queue = set()
    tutorOnDuty = dict()
    ticketsResolvedToday = 0
    totalTimeWeightingToday = 0
    prevEventTime = None
    n = 30
    
    # loop through all the events for the entire history of this queue from oldest to newest
    for i in events.index:
        tid = events["ticket_id"][i]

        # if the queue got cleared then clear our queue
        if prevEventTime and crossesBarrier(prevEventTime, events['time'][i]):
                queue.clear()
                tutorOnDuty.clear()
                ticketsResolvedToday = 0
                totalTimeWeightingToday = 0

        #remove all the tutors who haven't accepted a ticket in the last N minutes where N is ajustable
        removeStaleTutors(tutorOnDuty, events['time'][i], n)

        #Add this tutor to tutorOnDuty is the event them accepting a ticket
        if events['type'][i] == "ACCEPTED":
            tutorOnDuty[events['user_id'][i]] = events['time'][i]
            

        if events['type'][i] == "CREATED":
            tutor = None
            timeToResolve = None
            accepted = events['time'][i]
            for j in range(i, len(events.index)):
                # crossed barrier
                if crossesBarrier(events['time'][i], events['time'][j]):
                    break;
                # found accepted event
                if events['ticket_id'][i] == events['ticket_id'][j] and events['type'][j] == "ACCEPTED":
                    accepted = events['time'][j]
                    tutor = events['user_id'][j]
                    for k in range(j, len(events.index)):
                        if crossesBarrier(events['time'][j], events['time'][k]):
                            break;
                        if events['ticket_id'][j] == events['ticket_id'][k] and (events['type'][k] == "RESOLVED"):
                            timeToResolve = events['time'][k] - accepted
                    break;

            index += 1
            

            diff = accepted - events['time'][i]
            totalTimeWeightingToday += diff.total_seconds()
            ticketsResolvedToday += 1
            
            if ticketsResolvedToday > 0:
                avg_weight_time = totalTimeWeightingToday * 1.0 / ticketsResolvedToday
            else:
                avg_weight_time = 0

            tutors = [tutor for tutor in tutorOnDuty]
            results.loc[index] = [events['ticket_id'][i], len(queue),
                                  events['time'][i], accepted, diff,
                                  tutors, events['time'][i].to_pydatetime().date().weekday(),
                                  events['time'][i].to_pydatetime().time(), events['shortName'][i],
                                  avg_weight_time, ticketsResolvedToday, tutor, timeToResolve]


        if events['type'][i] == "CREATED" or events['type'][i] == "DEFERRED":
            if not tid in queue:
                queue.add(tid)
        elif events['type'][i] == "CANCELED" or events['type'][i] == "RESOLVED":
            if tid in queue:
                queue.remove(tid)

        prevEventTime = events['time'][i]

    results = results[results['diff'] != pandas.Timedelta('0 days')]
    results = results[results['diff'] < pandas.Timedelta('2 hours')]
    
    results.to_json(path_or_buf=destinationFolder + "/queue" + str(queueId) + ".json", orient='records')
    
    
    

In [28]:
def GetInfoFromAllQueues(minQueueId, maxQueueId, destinationFolder, clean):
    if clean == True:
        CleanAndMakeDir(destinationFolder)
    for i in range(minQueueId, maxQueueId + 1):
        GetInfoFromQueue(i, destinationFolder)

In [29]:
GetInfoFromAllQueues(1, 37, "QueueDataAll", True)

In [30]:
import json
import glob

result = []
for f in glob.glob("./QueueDataAll/*.json"):
    with open(f, "rb") as infile:
        result.append(json.load(infile))

with open("merged_file.json", "wb") as outfile:
     json.dump(result, outfile)